In [510]:
#1.寻找目标网站并分析结构
import requests
import re
import csv
from bs4 import BeautifulSoup
import json
import time
import random
import csv
header=['userid','username','userimg','create_date','residence','contact_num','rev_contact_num','book_do_num','book_wish_num','book_collect_num','movie_do_num','movie_wish_num','movie_collect_num','review_num','group_num','official_account']
with open('./crawler/doban.csv','w',encoding='utf-8') as f:
    f_csv=csv.writer(f)
    f_csv.writerow(header)
    
#定义爬虫的User Agent
user_agent=user_agent='Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.110 Safari/537.36'
cookie='viewed="30294837"; bid=akwf19Sj9vg; gr_user_id=e3f576be-96b7-4d23-8f81-5524f9ea9900; _vwo_uuid_v2=D9E83E42E6E6F8789BF8EDD449B9A896A|ae27072bc60e0a6c4907422d17bc74df; ll="118173"; __utmc=30149280; __utmz=30149280.1544341287.3.3.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; ap_v=0,6.0; __utma=30149280.1597611217.1543150549.1544346180.1544355300.5; _pk_ref.100001.8cb4=%5B%22%22%2C%22%22%2C1544357837%2C%22https%3A%2F%2Fmovie.douban.com%2Fsubject%2F24773958%2F%22%5D; _pk_ses.100001.8cb4=*; as="https://www.douban.com/people/lingrui1995/"; ps=y; dbcl2="174075919:oOlXd2Fnio4"; ck=ztF5; push_noty_num=0; push_doumail_num=0; __utmv=30149280.17407; douban-profile-remind=1; __yadk_uid=BLaoiyly2p2NOMHSBlhlWukjs9OBFF1x; douban-fav-remind=1; ct=y; __utmt=1; _pk_id.100001.8cb4=966b8455d2cbac96.1543646893.3.1544360056.1544341668.; __utmb=30149280.65.10.1544355300'
headers={'User-Agent':user_agent,'Cookie':cookie}

MAX=10
init_url='https://www.douban.com/people/12555460/'

def getUserInfo(user_url,count,last_user_url):
    print('开始爬取第{}个用户！！！！'.format(count))
    #使用requests发送请求
    user=requests.get(user_url,headers=headers)
    user.encoding='utf-8'
    soup=BeautifulSoup(user.text,'lxml')
    
    # 判断用户是否有效
    infobox=soup.find('div',class_='mn')
    if infobox:
        getUserInfo(last_user_url,count,last_user_url)
        return 0
        #isvalid=re.search(r'注销帐号',infobox.get_text()) #用户是否注销账
        #if isvalid:
        #    getUserInfo(last_user_url,count,last_user_url)
        #    return 0
    #获取用户基本信息
    all_info=[]
    #id
    userid=re.split(r'/',user_url)[-2]
    #用户名
    img=soup.find('div',class_='pic').find('img')
    if img:
        username=img.get('alt') 
    else:
        username=soup.find('div',class_='pic').find('a').get('title')
    #头像
    userimg=soup.find('div',class_='basic-info').find('img').get('src')
    
    #加入时间
    userinfo=soup.find('div',class_='user-info').find('div',class_='pl').get_text()
    create_date=re.search(r'(\d{4}-\d{2}-\d{2})',userinfo).group()
    #常居地
    residence1=soup.find('div',class_='user-info').find('a')
    residence=residence1.get_text() if residence1 else ''
    #是否是豆瓣官方运营帐号
    is_reason=soup.find('span',class_='reason')
    official_account=1 if is_reason else 0
    #书的数量
    book_do_num=0
    book_wish_num=0
    book_collect_num=0
    book=soup.find('div',id='book').find('span',class_='pl')
    if book:
        books=book.find_all('a')
        for i in range(len(books)):
            has_do=re.search('/do',books[i].get('href'))
            if has_do:  
                book_do_num=int(re.match(r'\d+',books[i].get_text()).group())
                continue
            has_wish=re.search('/wish',books[i].get('href'))
            if has_wish:
                book_wish_num=int(re.match(r'\d+',books[i].get_text()).group())
                continue   
            has_collect=re.search('/collect',books[i].get('href'))
            if has_collect:
                book_collect_num=int(re.match(r'\d+',books[i].get_text()).group())
                continue
    #电影数量
    movie_do_num=0
    movie_wish_num=0
    movie_collect_num=0
    movie=soup.find('div',id='movie').find('span',class_='pl')
    if movie:
        movies=movie.find_all('a')
        for i in range(len(movies)):
            has_do=re.search('/do',movies[i].get('href'))
            if has_do:  
                movie_do_num=int(re.match(r'\d+',movies[i].get_text()).group())
                continue
            has_wish=re.search('/wish',movies[i].get('href'))
            if has_wish:
                movie_wish_num=int(re.match(r'\d+',movies[i].get_text()).group())
                continue   
            has_collect=re.search('/collect',movies[i].get('href'))
            if has_collect:
                movie_collect_num=int(re.match(r'\d+',movies[i].get_text()).group())
                continue
    #评论数量
    review=soup.find('div',id='review').find('a')
    review_num=re.search(r'\d+',review.get_text()).group() if review else 0
    #常去小组数量
    group=soup.find('div',id='group').find('h2')
    group_num=re.findall('\((\d+)\)',group.get_text())[0] if group else 0
    
    #关注人数
    contact=soup.find('div',id='friend').find('a').get_text()
    contact_num=int(re.findall(r'成员(\d+)',contact)[0]) #关注人数
    
    #被关注人数
    rev_contact=soup.find('p',class_='rev-link').find('a').get_text()
    rev_contact_num=int(re.findall(r'被(\d+)',rev_contact)[0]) #被关注人数
    
    all_info.extend([userid,username,userimg,create_date,residence,contact_num,rev_contact_num,book_do_num,book_wish_num,book_collect_num,movie_do_num,movie_wish_num,movie_collect_num,review_num,group_num,official_account])
    print(all_info)
    if (count==0) | (user_url!=last_user_url):
        with open('./crawler/doban.csv','a',encoding='utf-8') as f:
            f_csv=csv.writer(f)
            f_csv.writerow(all_info)
    
    
    
    
    # 开始爬取下一个用户
    if contact_num & rev_contact_num:
        contact_or_rev=random.sample(range(0,2),1)[0]
    else:
        contact_or_rev = 0 if contact_num > 0 else 1
        
    print("关注人数",contact_num,"被关注人数",rev_contact_num)
        
    #进入关注人列表
    if contact_or_rev == 0:
        n=random.randint(0,contact_num-1) #随机获取列表中第n个用户 
        contact_text=requests.get(user_url+'/contacts',headers=headers).text
        
        contact_soup=BeautifulSoup(contact_text,'lxml')
        try:
            obu=contact_soup.find_all('dl',class_='obu')[n]
            obu_url=obu.find('a').get('href')
        except:
            print(n)
            print(contact_soup.find_all('dl',class_='obu'))
            
        print('关注人列表第{}个用户:'.format(n),obu_url)
    #进入被关注人列表
    else:
        n=random.randint(0,rev_contact_num-1) #随机获取列表中第n个用户 
        rev_contact_text=requests.get(user_url+'/rev_contacts?start='+str(n//70*70),headers=headers).text
        rev_contact_soup=BeautifulSoup(rev_contact_text,'lxml')
        try:
            obu=rev_contact_soup.find_all('dl',class_='obu')[n%70-1]
            obu_url=obu.find('a').get('href')
        except:
            print(n)
            print(rev_contact_soup.find_all('dl',class_='obu'))
        print('被关注人列表第{}个用户:'.format(n),obu_url)
    
    # 递归调用
    count+=1
    if count < MAX:
        getUserInfo(obu_url,count,user_url)
        
getUserInfo(init_url,0,init_url)


开始爬取第0个用户！！！！
['12555460', '莫吟思', 'https://img3.doubanio.com/icon/ul12555460-5.jpg', '2009-09-19', '北京', 38, 4318, 2, 6, 8, 0, 3, 33, '3', '62', 0]
关注人数 38 被关注人数 4318
关注人列表第4个用户: https://www.douban.com/people/58261194/
开始爬取第1个用户！！！！
['58261194', '书意', 'https://img3.doubanio.com/icon/ul58261194-4.jpg', '2012-02-03', '上海', 70, 1088, 0, 144, 95, 0, 1, 25, '4', '82', 0]
关注人数 70 被关注人数 1088
关注人列表第37个用户: https://www.douban.com/people/3790983/
开始爬取第2个用户！！！！
['3790983', '齐物秋水', 'https://img3.doubanio.com/icon/ul3790983-100.jpg', '2009-03-11', '北京', 312, 14504, 0, 161, 1045, 0, 10, 506, '304', '23', 0]
关注人数 312 被关注人数 14504
关注人列表第272个用户: https://www.douban.com/people/3319131/
开始爬取第3个用户！！！！
['3319131', '邝海炎', 'https://img3.doubanio.com/icon/ul3319131-2.jpg', '2008-12-09', '广东广州', 582, 1513, 242, 1992, 1227, 0, 4, 84, '46', '90', 0]
关注人数 582 被关注人数 1513
被关注人列表第31个用户: https://www.douban.com/people/179383126/
开始爬取第4个用户！！！！
['179383126', '颜轻', 'https://img3.doubanio.com/icon/ul179383126-1.jpg', '2018-0

In [156]:
rev_contact_text=requests.get('https://www.douban.com/people/70032148/',headers=headers).text
n=random.randint(0,-1) #随机获取列表中第n个用户 
rev_contact_soup=BeautifulSoup(rev_contact_text,'lxml')
obu=rev_contact_soup.find_all('dl',class_='obu')[n]
obu_url=obu.find('a').get('href')
print('被关注人列表第{}个用户:'.format(n),obu_url)

被关注人列表第7个用户: https://www.douban.com/people/46741762/


In [137]:
n=random.sample(range(0,3),1)
n

[2]

In [121]:
m=random.randint(0,3)
m

1

In [176]:
contact_text=requests.get('https://www.douban.com/people/85359370','/contacts',headers=headers).text

In [505]:
import datetime
user=requests.get('https://www.douban.com/people/180441686/',headers=headers)
user.encoding='utf-8'
soup=BeautifulSoup(user.text,'lxml')
#用户基本信息
#头像
userimg=soup.find('div',class_='basic-info').find('img').get('src')
#用户名
#username=soup.find('div',class_='pic').find('img').get('alt')
username=soup.find('div',class_='pic').find('a').get('title')
print(username)
#id
userinfo=soup.find('div',class_='user-info').find('div',class_='pl').get_text()
userid=re.match(r'\d+',userinfo).group()
#加入时间
create_date=re.search(r'(\d{4}-\d{2}-\d{2})',userinfo).group()
#常居地
residence1=soup.find('div',class_='user-info').find('a')
residence=residence1.get_text() if residence1 else ''
#是否是豆瓣官方运营帐号
is_reason=soup.find('span',class_='reason')
#reason=re.match(r'豆瓣官方运营帐号',soup.find('span',class_='reason').string)
official_account=1 if is_reason else 0
print(official_account)


#书的数量
book_do_num=0
book_wish_num=0
book_collect_num=0

book=soup.find('div',id='book').find('span',class_='pl').find_all('a')
for i in range(len(book)):
    has_do=re.search('/do',book[i].get('href'))
    if has_do:  
        book_do_num=re.match(r'\d+',book[i].get_text()).group()
        continue
    has_wish=re.search('/wish',book[i].get('href'))
    if has_wish:
        book_wish_num=re.match(r'\d+',book[i].get_text()).group()
        continue   
    has_collect=re.search('/collect',book[i].get('href'))
    if has_collect:
        book_collect_num=re.match(r'\d+',book[i].get_text()).group()
        continue


#电影数量
movie_do_num=0
movie_wish_num=0
movie_collect_num=0

movie=soup.find('div',id='movie').find('span',class_='pl').find_all('a')
for i in range(len(movie)):
    has_do=re.search('/do',movie[i].get('href'))
    if has_do:  
        movie_do_num=re.match(r'\d+',movie[i].get_text()).group()
        continue
    has_wish=re.search('/wish',movie[i].get('href'))
    if has_wish:
        movie_wish_num=re.match(r'\d+',movie[i].get_text()).group()
        continue   
    has_collect=re.search('/collect',movie[i].get('href'))
    if has_collect:
        movie_collect_num=re.match(r'\d+',movie[i].get_text()).group()
        continue

#评论数量
#review=soup.find('div',id='review').find('a').get_text()
#review_num=re.search(r'\d+',review).group() if review else 0
#常去小组数量

#group=soup.find('div',id='group').find('h2').get_text()
group=None
group_num=re.findall('\((\d+)\)',group)[0] if group else 0
#print(group_num)
all_info=[]
all_info.extend([userid,username,userimg,create_date,residence,book_do_num,book_wish_num,book_collect_num,movie_do_num,movie_wish_num,movie_collect_num,review_num,group_num])
#print(all_info)



#print(movie_do_num,movie_wish_num,movie_collect_num)

None
1


In [323]:
a=[1,2,0]
for i in range(3):
    if a[i]==1:
        print('a=1')
        continue
    if a[i]==2:
        a[i]=22
        print(a[i])
        continue
    if a[i]==3:
        continue

a=1
22


In [429]:
x=re.findall('\((\d+)\)','莫122吟思常去的小组(62)')
print(x[0])

62


In [350]:
has_do=None
print(has_do)
has_do=re.search('/do',book[1].get('href'))
print(book[1].get('href'))
print(has_do)

None
https://book.douban.com/people/12555460/wish
None


In [448]:
date='2018-12-09'
create_date=datetime.datetime.strptime(date, '%Y-%m-%d')
create_date

datetime.datetime(2018, 12, 9, 0, 0)

In [464]:
userid=re.split(r'/','https://www.douban.com/people/purpledreamer/')[-2]
print(userid)

purpledreamer


In [494]:

    
for i in range(3):
    row=[1+i,2+i,3+i]
    with open('./crawler/test.csv','a',encoding='utf-8') as f:
        f_csv=csv.writer(f)
        f_csv.writerow(row)  

In [493]:
header=['发文部门','标题','链接']
with open('./crawler/test.csv','w',encoding='utf-8') as f:
    f_csv=csv.writer(f)
    f_csv.writerow(header)